In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [1]:
#@title Part 1: Linear Regression




In [3]:
from google.colab import files
uploaded = files.upload()

Saving HW3 Data.csv to HW3 Data.csv


In [4]:
import io
df = pd.read_csv(io.BytesIO(uploaded['HW3 Data.csv']),parse_dates=['Date'])

In [5]:
df.drop(['Close'],inplace=True,axis=1)
df['Date']=pd.to_datetime(df['Date'])
df= df.set_index(df['Date'])
df= df.sort_index()
df_train = df['2017':'2020']
df_test  = df['2021':]
df_train.head()

,Date,Return
Date,,
2017-01-03,2017-01-03,0.0000
2017-01-04,2017-01-04,0.0059
2017-01-05,2017-01-05,-0.0008
2017-01-06,2017-01-06,0.0036
2017-01-09,2017-01-09,-0.0033


In [6]:
df_test.head()

,Date,Return
Date,,
2021-01-04,2021-01-04,-0.0136
2021-01-05,2021-01-05,0.0069
2021-01-06,2021-01-06,0.0060
2021-01-07,2021-01-07,0.0149
2021-01-08,2021-01-08,0.0057


In [7]:
train = df_train['Return'].to_list()
test = df_test['Return'].to_list()

In [8]:
def preprocess(data, length=6):
    n = len(data)
    d = []
    for i in range(0, n-length+1):
        d1 = data[i: i+length]
        d.append(d1)
    d = np.array(d)
    x = d[:, :-1]
    y = d[:, -1]
    return x, y

In [17]:
size = [i for i in range(1, 151)]

ltrain = []
ltest = []
intercepts = []
coeff = []

for length in size:
    x, y = preprocess(train, length=length+1)
    model = LinearRegression().fit(x, y)
    ypred = model.predict(x)
    loss_train = mean_squared_error(y, ypred)
    x, y = preprocess(test, length=length+1)
    ypred = model.predict(x)
    loss_test = mean_squared_error(y, ypred)
    ltrain.append(loss_train)
    ltest.append(loss_test)
    intercepts.append(model.intercept_)
    coeff.append(model.coef_)

In [19]:
# find the best N which reaches minimal test loss
minloss = ltest.index(min(ltest))
b=minloss+1

print(f'The best N is {b} and the loss is {min(ltest)}')

The best N is 140 and the loss is 3.851008376980854e-05


In [ ]:
#@title Part 2: DNN


In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU
from sklearn.preprocessing import StandardScaler

In [29]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

x_train,y_train=preprocess(train)
x_test,y_test=preprocess(test)


In [30]:
def create_new_model(layers,activation):
  model=Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=x_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  model.add(Dense(units=1,kernel_initializer='glorot_uniform', activation='sigmoid'))
  model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
  return model



In [31]:
model= KerasClassifier(build_fn=create_new_model,verbose=0)

In [33]:
layers=[[b],[b]*2,[b]*3,[b]*4,[b]*5,[b]*6,[b]*7,[b]*8,[b]*9,[b]*10]
activations=['sigmoid','relu']
para_grid=dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
g=GridSearchCV(estimator=model,param_grid=para_grid,cv=5)
g_result=g.fit(x_train,y_train)

In [34]:
print(g_result.best_score_,g_result.best_params_)

0.0019999999552965165 {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 30, 'layers': [140]}


In [35]:
y_hat=g.predict(x_test)

In [36]:
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(y_test,y_hat)
print("Mean Squared Error For DNN is:",MSE)

Mean Squared Error For DNN is: 0.009431250372670808
